In [1]:
from sage.all import *

# Параметры кода
q = 2  # Порядок поля
k_0 = 1  # Степень полинома
n = q + 1  # Длина кода
#N = (q**2 + q + 1) * (q + 1)  # Длина кода Рида-Соломона
F = GF(q)  # Конечное поле
R.<x, y, z> = PolynomialRing(F)  # Для использования x, y, z для однородных полиномов
k = k_0 + 1

#print(f"N = {N}")
# Генерация проективной плоскости
def generate_projective_plane(F):
    points = []
    lines = []
    for x in F:
        for y in F:
            for z in F:
                if (x, y, z) != (0, 0, 0):  # Исключаем точку (0, 0, 0)
                    is_unique = True
                    for a in F:
                        if a != 0:
                            # Масштабируем точку на элементы поля F
                            scaled_point = (a * x, a * y, a * z)
                            if scaled_point in points:
                                is_unique = False
                                break
                    if is_unique:
                        points.append((x, y, z))  # Добавляем точку в список
                        lines.append((x, y, z))   # Прямую тоже
    return points, lines

# Генерация точек и прямых
points, lines = generate_projective_plane(F)
print("Точки проективной плоскости:")
print(points)
print("Прямые проективной плоскости:")
print(lines)

# Матрица инцидентности
def incidence_matrix(points, lines, F):
    incidence = Matrix(F, len(points), len(lines))
    edges = []
    edge_numbers = {}
    for i, point in enumerate(points):
        for j, line in enumerate(lines):
            if sum(F(point[k]) * F(line[k]) for k in range(3)) == 0:  # Проверка инцидентности
                incidence[i, j] = 1
                edge_numbers[(i, j)] = len(edges)
                edges.append((i, j))
    return incidence, edges, edge_numbers

# Создаем матрицу инцидентности
I, edges, edge_numbers = incidence_matrix(points, lines, F)

def pow_f(x, y):
    if x == 0:
        return F(1) if y == 0 else F(0)
    return x ** y

# Функция для генерации проверочной матрицы для прямой с учётом инцидентности
def parity_check_matrix_for_line(N, n, k, line, points, incidence_matrix, edge_numbers):
    matrix_rows = []
    print(f"matrix for line({line[0]},{line[1]},{line[2]}), n = {n}, k = {k}")
    for i in range(0, n - k):  # Генерация строк для полиномов
        for j in range(0, n - k):
            if i + j >= n-k:
                continue
            row = [0]*N
            print(f"row({i},{j})")
            # Для каждой точки, инцидентной данной прямой
            for s, point in enumerate(points):
                if incidence_matrix[s, lines.index(line)] == 1:  # Проверяем инцидентность
                    print(f"point({point[0]},{point[1]},{point[2]})")
                    val = pow_f(point[0], i) * pow_f(F(point[1]), j) * pow_f(F(point[2]),n - k - 1 - i - j)
                    row[edge_numbers[(s, lines.index(line))]] = val
                    #row.append(pow_f(point[0], i) * pow_f(F(point[1]), j) * pow_f(F(point[2]),n - k - i - j))  # Генерация полинома
                #else:
                #    row.append(0)  # Если точка не инцидентна прямой, то ставим 0
            matrix_rows.append(row)
    print(matrix_rows)
    # удалим линейно зависимые строки
    rk = Matrix(matrix_rows).rank()
    print(rk)
    matrix_rows = Matrix(matrix_rows).echelon_form().rows()[:rk]
    return Matrix(F, matrix_rows)

# Функция для генерации проверочной матрицы для точки с учётом инцидентности
def parity_check_matrix_for_point(N, n, k, point, lines, incidence_matrix, edge_numbers):
    matrix_rows = []
    for i in range(0, n - k):  # Генерация строк для полиномов
        for j in range(0, n - k):
            if i + j >= n-k:
                continue
            row = [0] * N
            # Для каждой прямой, инцидентной данной точке
            for s, line in enumerate(lines):
                if incidence_matrix[points.index(point), s] == 1:  # Проверяем инцидентность
                    val = pow_f(line[0], i) * pow_f(F(line[1]), j) * pow_f(F(line[2]), n - k - 1 - i - j)  # Генерация полинома
                    row[edge_numbers[(points.index(point), lines.index(line))]] = val
                #else:
                #    row.append(0)  # Если точка не инцидентна прямой, то ставим 0
            matrix_rows.append(row)
    # удалим линейно зависимые строки
    rk = Matrix(matrix_rows).rank()
    print(rk)
    matrix_rows = Matrix(matrix_rows).echelon_form().rows()[:rk]
    return Matrix(F, matrix_rows)

# Функция для генерации матрицы Таннера
def tanner_matrix(incidence_matrix, n, k_0, points, lines, edge_numbers):
    tanner_matrix = [] #Matrix(F, 2 * (q**2 + q + 1) * (n - k), (q + 1) * (q**2 + q + 1))
    N = (q**2 + q + 1) * (q + 1)
    k = k_0 + 1
    # Заполнение верхней части матрицы для точек
    for i in range(q**2 + q + 1):  # Это количество точек
        #for j in range(q + 1):  # Это количество прямых, с которыми точка инцидентна
        #    if incidence_matrix[i, j] == 1:
        point_matrix = parity_check_matrix_for_point(N, n, k, points[i], lines, incidence_matrix, edge_numbers)
        tanner_matrix += point_matrix
                # Заполняем строку для точки в матрице Таннера
                #for col in range(min(q + 1, point_matrix.ncols())):  # Ограничиваем диапазон для col
                #    if col < point_matrix.ncols() and j < point_matrix.nrows():
                #        tanner_matrix[i, col] = point_matrix[col, j]

    # Заполнение нижней части матрицы для прямых
    for i in range(q**2 + q + 1):  # Это количество прямых
        #for j in range(q + 1):  # Это количество точек, инцидентных прямой
            #if incidence_matrix[i, j] == 1:
        line_matrix = parity_check_matrix_for_line(N, n, k, lines[i], points, incidence_matrix, edge_numbers)
        tanner_matrix += line_matrix
                # Заполняем строку для прямой в матрице Таннера
                #for col in range(min(q + 1, line_matrix.ncols())):  # Ограничиваем диапазон для col
                #    if col < line_matrix.ncols() and j < line_matrix.nrows():
                #        tanner_matrix[q**2 + q + 1 + i, col] = line_matrix[col, j]

    return Matrix(F, tanner_matrix)

# Генерация матрицы Таннера
T = tanner_matrix(I, n, k_0, points, lines, edge_numbers)
rk = Matrix(T).rank()
print("Ранг матрицы Таннера:")
print(rk)
# Выводим результат

#print("Матрица Таннера:")
#print(T)
#print(T.dimensions())



Точки проективной плоскости:
[(0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
Прямые проективной плоскости:
[(0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
1
1
1
1
1
1
1
matrix for line(0,0,1), n = 3, k = 2
row(0,0)
point(0,1,0)
point(1,0,0)
point(1,1,0)
[[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]
1
matrix for line(0,1,0), n = 3, k = 2
row(0,0)
point(0,0,1)
point(1,0,0)
point(1,0,1)
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]
1
matrix for line(0,1,1), n = 3, k = 2
row(0,0)
point(0,1,1)
point(1,0,0)
point(1,1,1)
[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
1
matrix for line(1,0,0), n = 3, k = 2
row(0,0)
point(0,0,1)
point(0,1,0)
point(0,1,1)
[[0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
1
matrix for line(1,0,1), n = 3, k = 2
row(0,0)
point(0,1,0)
point(1,0,1)
point(1,1,1)
[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]]
1
matrix